In [1]:
# generate data for testing acc@10
# pick the first 100 ligands from the test_set
# pair all proteins with all ligands
# (100x100 pairs)
# (total of 10,000 pairs...)

In [1]:

from pathlib import Path
import numpy as np
import pickle as pk
from multiprocessing import Pool
import pandas as pd
from tqdm import tqdm
from htmd.ui import *
from htmd.molecule.voxeldescriptors import *
from multiprocessing import Pool 

%load_ext autoreload
%autoreload 2
%cd ..

#Supress default INFO logging

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)


/home/alex/Desktop/drug-discovery
ffevaluate module is in beta version

Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. 
https://dx.doi.org/10.1021/acs.jctc.6b00049
Documentation: http://software.acellera.com/
To update: conda update htmd -c acellera -c psi4

You are on the latest HTMD version (1.13.9).



In [2]:
test_size = 300
base_dest = './processed_data/test_acc10'
Path(base_dest).mkdir(exist_ok=True, parents=True)
df = pd.read_csv('./data/csv/test_lig_pro_pairs.csv')
idxs = sorted(list(df.lig_id.unique()))
idxs = idxs[:test_size]


In [4]:
# generate the data

for lig_idx in tqdm(idxs):
    lig_src = './processed_data/ligands/' + str(lig_idx).zfill(4) + '.pk'
    with open(lig_src, 'rb') as f:
        lig_features, centers = pk.load(f)
    
    for pro_idx in idxs:
        dest = base_dest + '/' + str(pro_idx).zfill(4) + '_pro_' + str(lig_idx).zfill(4) + '_lig.npy'
        
        protein_path = './data/filtered_data/'+str(pro_idx).zfill(4)+'_pro_cg.pdb'
        

        combined = np.concatenate((pro_features[:,:,:,None], lig_features[:,:,:,None]), axis=3)
        combined = combined.astype(np.float32)

        np.save(dest, combined)

  0%|          | 0/300 [00:00<?, ?it/s]/home/alex/miniconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \
100%|██████████| 300/300 [2:10:41<00:00, 25.68s/it]


In [5]:
# generate the csv
dests = []
ligs = []
proteins = []
score = []
for lig_idx in tqdm(idxs):
    lig_src = './processed_data/ligands/' + str(lig_idx).zfill(4) + '.pk'
    
    for pro_idx in idxs:
        dest = base_dest + '/' + str(pro_idx).zfill(4) + '_pro_' + str(lig_idx).zfill(4) + '_lig.npy'
        
        dests.append(dest)
        ligs.append(lig_idx)
        proteins.append(pro_idx)
        score.append(1 if lig_idx == pro_idx else 0)

100%|██████████| 300/300 [00:00<00:00, 1450.16it/s]


In [7]:
csv_dest = f'./data/csv/test_acc10_{test_size}.csv'
pro_lig_record = pd.DataFrame({'lig_id': ligs, 'pro_id': proteins, 
                               'dest': dests, 'score': score}, index=None)
pro_lig_record.to_csv(csv_dest, index=None)